## Name 
#### Yingtong Lin

## Research Question / Hypothesis

#### Using supervised learning, predict a user's rating on animes based on features.

## Packages import

In [ ]:
! pip install mord

  Created wheel for mord: filename=mord-0.6-cp37-none-any.whl size=6007 sha256=515e2cdc354067e1371a31e46a87cd0bf6ecca7d77522da123f50b9fd30498d6
  Stored in directory: /root/.cache/pip/wheels/98/14/b2/244c2cec93a0c6edb29b488bd6b2710ded7e9d457033b86366
Successfully built mord


In [ ]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.impute import SimpleImputer
from   sklearn.pipeline           import Pipeline
from   sklearn.preprocessing      import *
from sklearn.compose import ColumnTransformer
from sklearn.linear_model  import LinearRegression, Lasso, Ridge, ElasticNet, HuberRegressor, BayesianRidge
from sklearn.metrics       import mean_squared_error
from sklearn.base            import BaseEstimator
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model    import LogisticRegression, RidgeClassifier
from sklearn.ensemble        import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import mord
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

## Load Data

In [ ]:
df = pd.read_csv('new_anime.csv')

In [ ]:
df.head()

,anime_id,name,type,episodes,rating,members,user_id,anime_id.1,rating.1,action,adventure,cars,comedy,dementia,demons,drama,ecchi,fantasy,game,harem,historical,horror,josei,kids,magic,martial arts,mecha,military,music,mystery,parody,police,psychological,romance,samurai,school,sci-fi,seinen,shoujo,shoujo ai,shounen,slice of life,space,sports,super power,supernatural,thriller,vampire
0,9253,Steins;Gate,TV,24,9.17,673572,5,6,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,9969,Gintama&#039;,TV,51,9.16,151266,5,15,6,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
2,15335,Gintama Movie: Kanketsu-hen - Yorozuya yo Eien...,Movie,1,9.10,72534,5,17,6,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
3,15417,Gintama&#039;: Enchousen,TV,13,9.11,81109,5,18,6,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
4,918,Gintama,TV,201,9.04,336376,5,20,6,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
df.shape

(467, 48)

## Check number of missing values

In [ ]:
df.isna().sum()

anime_id         0
name             0
type             0
episodes         0
rating           0
members          0
user_id          0
anime_id.1       0
rating.1         0
action           0
adventure        0
cars             0
comedy           0
dementia         0
demons           0
drama            0
ecchi            0
fantasy          0
game             0
harem            0
historical       0
horror           0
josei            0
kids             0
magic            0
martial arts     0
mecha            0
military         0
music            0
mystery          0
parody           0
police           0
psychological    0
romance          0
samurai          0
school           0
sci-fi           0
seinen           0
shoujo           0
shoujo ai        0
shounen          0
slice of life    0
space            0
sports           0
super power      0
supernatural     0
thriller         0
vampire          0
dtype: int64

## Split dataframe to X and y

drop anime_id, anime_name, user_id: not anime features, drop rating, members: unpredictive, not correlated to user rating

In [ ]:
X = df.drop('rating.1', axis=1).drop(columns=['anime_id.1', 'anime_id', 'user_id','name', 'rating', 'members'])

In [ ]:
X.head(3)

,type,episodes,action,adventure,cars,comedy,dementia,demons,drama,ecchi,fantasy,game,harem,historical,horror,josei,kids,magic,martial arts,mecha,military,music,mystery,parody,police,psychological,romance,samurai,school,sci-fi,seinen,shoujo,shoujo ai,shounen,slice of life,space,sports,super power,supernatural,thriller,vampire
0,TV,24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,TV,51,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0
2,Movie,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0


In [ ]:
X.columns

Index(['type', 'episodes', 'action', 'adventure', 'cars', 'comedy', 'dementia',
       'demons', 'drama', 'ecchi', 'fantasy', 'game', 'harem', 'historical',
       'horror', 'josei', 'kids', 'magic', 'martial arts', 'mecha', 'military',
       'music', 'mystery', 'parody', 'police', 'psychological', 'romance',
       'samurai', 'school', 'sci-fi', 'seinen', 'shoujo', 'shoujo ai',
       'shounen', 'slice of life', 'space', 'sports', 'super power',
       'supernatural', 'thriller', 'vampire'],
      dtype='object')

In [ ]:
y = df['rating.1']
y

0      8
1      6
2      6
3      6
4      6
      ..
462   -1
463    7
464   -1
465    9
466    7
Name: rating.1, Length: 467, dtype: int64

## Column Transformer

In [ ]:
genres = ['action', 'adventure', 'cars',
       'comedy', 'dementia', 'demons', 'drama', 'ecchi', 'fantasy', 'game',
       'harem', 'historical', 'horror', 'josei', 'kids', 'magic',
       'martial arts', 'mecha', 'military', 'music', 'mystery', 'parody',
       'police', 'psychological', 'romance', 'samurai', 'school', 'sci-fi',
       'seinen', 'shoujo', 'shoujo ai', 'shounen', 'slice of life', 'space',
       'sports', 'super power', 'supernatural', 'thriller', 'vampire']

In [ ]:
cat_col = ['type'] 
con_col = ['episodes'] + genres

In [ ]:
# Standardize features by removing the mean and scaling to unit variance
continuous_pipe = Pipeline([('scaler', StandardScaler())]) 
# Encode categorical features as a one-hot numeric array.
categorical_pipe = Pipeline([('OHE', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
preprocessing = ColumnTransformer([('categorical', categorical_pipe, cat_col),
                                   ('continuous', continuous_pipe, con_col)])

## Data Transformation Pipeline

In [ ]:
class DummyEstimator(BaseEstimator):
    "Pass through class, methods are present but do nothing."
    def fit(self): pass
    def score(self): pass

In [ ]:
pipe = Pipeline([('preprocessing', preprocessing), 
                 ('clf', DummyEstimator())])

## Multiclass classification by Grid Search

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y.astype('str'))

In [ ]:
search_space = [{'clf': [LogisticRegression()],
                'clf__penalty': ['l1', 'l2', 'elasticnet', 'none'], # Used to specify the norm used in the penalization.
                'clf__C': np.logspace(0, 4, 10), # Tolerance for stopping criteria.
                'clf__multi_class': ['multinomial','auto'], 
                'clf__class_weight': ['balanced','None'], # Weights associated with classes in the form {class_label: weight}
                'clf__solver' : ['newton-cg','lbfgs', 'liblinear', 'sag', 'saga']}, # Algorithm to use in the optimization problem.
                
               {'clf': [RandomForestClassifier()], 
                'clf__criterion': ['gini', 'entropy'], # The function to measure the quality of a split.
                'clf__n_estimators': [50,100,150,200], # The number of trees in the forest.
                'clf__max_features': ['auto','sqrt','log2'], # The number of features to consider when looking for the best split
                'clf__max_depth': [20,50,100,200], # The maximum depth of the tree.
                'clf__class_weight' : ['balanced','balanced_subsample']}, # Weights associated with classes in the form {class_label: weight}.
                
               {'clf': [ExtraTreesClassifier()],
                'clf__criterion': ['gini', 'entropy'], 
                'clf__n_estimators': [50,100,150,200],
                'clf__max_features': ['auto','sqrt','log2'],
                'clf__max_depth': [20,50,100,200],
                'clf__class_weight' : ['balanced','balanced_subsample']}]

In [ ]:
clf_algos_rand = GridSearchCV(estimator=pipe, 
                                    param_grid=search_space,
                                    n_jobs=-1,
                                    verbose=1)

clf_algos_rand.fit(X_train, y_train)
clf_algos_rand.best_estimator_

Fitting 5 folds for each of 1184 candidates, totalling 5920 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 1170 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 1720 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 2370 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 3120 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 3970 tasks      | elapsed:   27.5s
[Parallel(n_jobs=-1)]: Done 4920 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 5920 out of 5920 | elapsed:   49.3s finished


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('categorical',
                                                  Pipeline(memory=None,
                                                           steps=[('OHE',
                                                                   OneHotEncoder(categories='auto',
                                                                                 drop=None,
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=True))],
                                          

In [ ]:
y_pred = clf_algos_rand.predict(X_validation)

### F1 score

The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. 

In [ ]:
 f1_score(y_validation, y_pred, average='weighted')

0.1420419609248868

### Confusion Matrix

Compute confusion matrix to evaluate the accuracy of a classification.

In [ ]:
By definition a confusion matrix  is such that  is equal to the number of observations known to be in group  and predicted to be in group .
confusion_matirx = pd.DataFrame(confusion_matrix(y_true=y_validation, y_pred=y_pred))
confusion_matirx

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,0,0,0,1,0,0
1,0,3,0,2,1,0,4,2,2,1,1
2,0,0,0,0,0,0,0,0,0,0,0
3,1,2,0,4,2,0,4,0,1,0,1
4,0,4,0,1,0,0,5,1,2,3,0
5,0,2,0,2,2,1,0,1,1,1,0
6,0,4,0,0,1,0,4,3,3,1,0
7,0,5,0,1,1,0,5,1,1,2,1
8,0,3,0,0,1,1,4,1,3,1,0
9,0,1,1,2,2,1,1,1,0,1,1


###### Build a text report showing the main classification metrics.

In [ ]:
print(classification_report(y_validation, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         1
           1       0.12      0.19      0.15        16
          10       0.00      0.00      0.00         0
           2       0.33      0.27      0.30        15
           3       0.00      0.00      0.00        16
           4       0.33      0.10      0.15        10
           5       0.15      0.25      0.19        16
           6       0.10      0.06      0.07        17
           7       0.21      0.21      0.21        14
           8       0.09      0.09      0.09        11
           9       0.00      0.00      0.00         1

    accuracy                           0.15       117
   macro avg       0.12      0.11      0.11       117
weighted avg       0.16      0.15      0.14       117



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Ordinal Regression using Mord

In [ ]:
X_ord =  preprocessing.fit_transform(X)

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_ord, y)

In [ ]:
# credit: https://github.com/pbhatnagar3/icgauge/blob/master/icgauge/training_functions.py
def fit_classifier_with_crossvalidation(X, y, basemod, cv, param_grid, 
                                        scoring='f1_macro', verbose=False):   
    crossvalidator = GridSearchCV(basemod, param_grid, cv=cv, scoring=scoring)
    crossvalidator.fit(X, y)
    print("Best params", crossvalidator.best_params_)
    print("Best score: %0.03f" % crossvalidator.best_score_)
    return crossvalidator.best_estimator_

def fit_logistic_or_with_crossvalidation(X, y):
    basemod = mord.OrdinalRidge()
    cv = 5
    param_grid = {'fit_intercept': [True, False], 
                  'alpha': [0.2, 0.4, 0.6, 0.8, 1.0, 2.0, 3.0],
                  'normalize': [True, False]}    
    return fit_classifier_with_crossvalidation(X, y, basemod, cv, param_grid,
                                               verbose=False)

In [ ]:
fit_logistic_or_with_crossvalidation(X_train, y_train)

Best params {'alpha': 3.0, 'fit_intercept': False, 'normalize': True}
Best score: 0.069


OrdinalRidge(alpha=3.0, copy_X=True, fit_intercept=False, max_iter=None,
             normalize=True, random_state=None, solver='auto', tol=0.001)

In [ ]:
ord = mord.OrdinalRidge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
             normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
ord.fit(X_train, y_train)

OrdinalRidge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
             normalize=False, random_state=None, solver='auto', tol=0.001)

In [ ]:
ord_pred = ord.predict(X_validation)

In [ ]:
f1_score(y_validation, ord_pred, average='weighted')

0.07518491327611157

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c0b1d54b-27ab-4f4b-8849-d33cfa555ed2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>